In [1]:
'''
Title: Birgam LLM Model
Author: Siddharth Sharma
Date: Aug, 24,2024 -- PST
Discription: This notbook contains basics for the LLM Project. - A Bigram Language Model.
--
To_deploy: Streamlit
--

References:
Step by Step  vedio:
https://www.freecodecamp.org/news/how-to-build-a-large-language-model-from-scratch-using-python/ To Follow this video
https://github.com/ajitsingh98/LLMs-From-Scratch-In-PyTorch

Book / Reading:
https://github.com/ManalAbumelhaa/LLM-LLMs-from-scratch

'''

'\nTitle:\nAuthor:\nDate\nDiscription: This notbook contains basics for the LLM Project.\nTo_deploy: Streamlit\n--\n\nReferences:\nStep by Step  vedio:\nhttps://www.freecodecamp.org/news/how-to-build-a-large-language-model-from-scratch-using-python/ To Follow this video\nhttps://github.com/ajitsingh98/LLMs-From-Scratch-In-PyTorch\n\nBook / Reading:\nhttps://github.com/ManalAbumelhaa/LLM-LLMs-from-scratch\n\n'

In [2]:
# iinstall the compression algo
!pip install pylzma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 44.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylzma: filename=pylzma-0.5.0-cp310-cp310-linux_x86_64.whl size=222315 sha256=f798af9174b28a68c707fe2c49b03281ea851e6d121d8aeb05614d95cb1b8569
  Stored in directory: /root/.cache/pip/wheels/74/c9/02/91112815e838f544c1d46fda071241e454694579d022751d2b
Successfully built pylzma


In [16]:
import pylzma
import torch
import torch.nn as nn
from torch.nn import functional as F

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

In [4]:
from google.colab import drive
drive.mount ('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
with open ("/content/drive/MyDrive/Colab Notebooks/From Scratch/Data Sets/wizard_of_oz.txt" , 'r' , encoding = 'utf-8') as f:
  text = f.read()

In [6]:
print(text[:300]) # if we do simply text[:300] it will not be visually appealing as it will also have the newline characters.


Title: The Wonderful Wizard of Oz

Author: L. Frank Baum

Release date: February 1, 1993 [eBook #55]
                Most recently updated: March 30, 2021

Language: English



*** START OF THE PROJECT GUTENBERG EBOOK THE WONDERFUL WIZARD OF OZ ***

[Illustration]




The Wonderful Wizard of Oz

by


In [7]:
len(text)

208087

We now need to build a tokenizer, wherein we will see how many characters do the whole text contains and this tokenizer will contain of a encoder and a decoder, each character will be assigned a number or a  (HASH)


This is a character level tokenizer,

there are other - sub word level, sentence level and word level tokenizer's

In [9]:
chars = sorted(set(text))
print(chars) # this contains all the different characters present in the text.
print(len(chars))

['\n', ' ', '!', '#', '&', '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '5', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”']
79


In [10]:
string_to_int = {ch:i for i,ch in enumerate(chars)} # in dict format
int_to_string = {i:ch for i,ch in enumerate(chars)}

# print(string_to_int)
# print(int_to_string)

encode = lambda s:[string_to_int[c] for c in s] # keeping it simple here, but better encoding  will make the custom code more secure...
decode = lambda l: ''.join([int_to_string[i] for i in l])

# sample to see if the intended code working or not
print(encode("testing"))
print(encode("Testing"))

print(decode(encode("Testing")))


[67, 52, 66, 67, 56, 61, 54]
[39, 52, 66, 67, 56, 61, 54]
Testing


torch.long - we are going to create a tensor and we will have the tensor conaitn super long integers

we will be storign our data in tensors now.

In [11]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:50])

tensor([ 0, 39, 56, 67, 59, 52, 17,  1, 39, 55, 52,  1, 42, 62, 61, 51, 52, 65,
        53, 68, 59,  1, 42, 56, 73, 48, 65, 51,  1, 62, 53,  1, 34, 73,  0,  0,
        20, 68, 67, 55, 62, 65, 17,  1, 31, 10,  1, 25, 65, 48])


Now once the data is defined in tensor's we will be splitting the data into training and testing split. the text corpus now. (70/30) split since the data corpus is small.

Link of the chat: https://chatgpt.com/share/6d4cb86a-e766-4796-8c74-63381191184d

***Bigram Models***

 bigram model is a type of statistical language model used in natural language processing (NLP) that predicts the probability of a word given the previous word in a sequence. In simpler terms, it considers pairs of consecutive words (known as "bigrams") to understand the likelihood of a word following another in a sentence.

How it works:
Sequence of Words: The bigram model breaks down a text into pairs of consecutive words.
For example, in the sentence "I am learning NLP," the bigrams would be ("I", "am"), ("am", "learning"), and ("learning", "NLP").
Probability Calculation: The model calculates the probability of a word given the previous word,


appears as the first word in any bigram in the text.
Application: This model is used to predict the next word in a sequence, generate text, or assess the likelihood of a given sequence of words. It's relatively simple but provides a foundational understanding of word dependencies in text.
Limitations:
Context: The bigram model only considers the previous word, so it doesn't capture long-range dependencies or context beyond one word back.
Data Sparsity: If certain bigrams are rare or unseen in the training data, the model might struggle to predict them accurately.
Overall, while the bigram model is a fundamental and straightforward approach, it is often extended to more complex models like trigrams (which consider three-word sequences) or even more sophisticated models like neural networks that capture broader context.



In [12]:
# creating a test train split:
# 70 percent training
threshold= 0.7*len(data)

train_text = data[:int(threshold)]
test_text = data[int(threshold):]


In [32]:
block_size = 6

x = train_text[:block_size]
y = train_text[1:block_size+1] # increamenting the character by 1, to see whether if it changes the proba of the otucomes.

for block_word in range (block_size):
  context = x[:block_word +1] # the context will keep on increasing, as the for loops progress, at the end of the loop the context will be the same size of 8 block words (here) (or the prompt data])
  target = y[block_word] #
  print(f"when input is {context} the target: {target}")

when input is tensor([0]) the target: 39
when input is tensor([ 0, 39]) the target: 56
when input is tensor([ 0, 39, 56]) the target: 67
when input is tensor([ 0, 39, 56, 67]) the target: 59
when input is tensor([ 0, 39, 56, 67, 59]) the target: 52
when input is tensor([ 0, 39, 56, 67, 59, 52]) the target: 17


Now that the block size is defined as 8 and the increment as 1, we  want to scale it up and hence we are wanting to process this in batches: (easist wyay to scale up and perform deep learningnlp tasks)

I'd be using the basic GPU T4 - Google collab easily available for a bit of faster processing, else CPU

In [15]:
# A simple preformance metric.
start_time = time.time()
end_time = time.time()
elapsed_time = end_time - start_time # if we want to see the process
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 2.86102294921875e-05 seconds


In [17]:
tensor_1 = torch.tensor([1.,2.,3.])

# get probas / Apply softmax
softmax_output = F.softmax(tensor_1, dim=0)

# NOTE: Why dim=0?
# In this context, dim=0 is used because tensor_1 is a 1-dimensional tensor.
# If it were a multi-dimensional tensor, dim would control along which dimension the softmax operation is applied.
# Since there's only one dimension here, dim=0 is appropriate.
# If you were working with a 2D tensor (e.g., a matrix), specifying dim=0 would apply the softmax across columns,
# while dim = 1 would apply it across rows.


print(softmax_output)

tensor([0.0900, 0.2447, 0.6652])


Todo: Learn a bit more about the nn.embeddings layering nd how this embedding is done.

In [24]:
# Initialize an embedding layer
vocab_size = 10000
embedding_dim = 100
embedding = nn.Embedding(vocab_size, embedding_dim)
# Create some input indices
input_indices = torch.LongTensor([1,5, 3, 2])
# Apply the embedding layer

embedded_output = embedding(input_indices)
# The output will be a tensor of shape (4, 100), where 4 is the number of input
# and 100 is the dimensionality of the embedding vectors

print(embedded_output.shape)
embedded_output_with_nan = torch.where(embedded_output > 1, embedded_output, torch.tensor(np.nan))
print(embedded_output_with_nan)

torch.Size([4, 100])
tensor([[   nan,    nan,    nan,    nan,    nan,    nan, 1.2166, 1.0423,    nan,
            nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
            nan, 2.8008,    nan,    nan,    nan,    nan,    nan,    nan, 1.6144,
            nan, 1.4763,    nan,    nan, 1.3010,    nan,    nan,    nan, 1.0067,
            nan,    nan,    nan,    nan,    nan,    nan, 1.0816,    nan,    nan,
            nan, 1.3635, 1.2689, 1.5447,    nan,    nan,    nan, 1.3808,    nan,
            nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan, 2.4802,
            nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
            nan,    nan,    nan,    nan, 1.6646,    nan, 2.3350,    nan, 3.1220,
            nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan, 1.0483,
            nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan, 1.6848,
            nan],
        [   nan,    nan,    nan,    nan,    nan,    nan,    nan, 1.343

Matmul

In [27]:
# a = torch.tensor([[1,2],[3,4],[5,6]])
# b = torch.tensor([[2,8,4],[10,11,12]])
# print(a @ b)
# @ or matmul is the same {kinda}, but in pytorch you can not multiply int and floats.

tensor([[ 22,  30,  28],
        [ 46,  68,  60],
        [ 70, 106,  92]])


We will now make a bigram model, and we will be keeping a size of 8 for the block. this is a character level bigram model and here the increament of the block "characters" will be by 1 (block size remaining the same)

In [14]:
# making different env / Choosing Cuda for batch processing:
device = 'cuda' if torch.cuda.is_available() else 'cpu' # just checks if gpu is available or not (here T4)
print(device)

# Constants:
# hyper parameters
block_size = 8
batch_size = 4


In [38]:
# Batch code:

def get_batch(split):
  data = train_text if split == 'train' else test_text
  i_x = torch.randint(len(data) - block_size, (batch_size,))
  # print(len(data) - block_size, (batch_size,))
  # print(len(data)) # why ?
  print(i_x)
  # print(i_x.shape)
  x = torch.stack([data[i:i+block_size] for i in i_x])
  y = torch.stack([data[i+1:i+block_size+1] for i in i_x])
  x,y = x.to(device), y.to(device) # Cuda here -  to process the data in batches.
  # in CPU the processing would be in sequence whic will take a lot of time and the
  # context windows might also suffer, in better / advanced models.
  return x,y



In [39]:
  # print(train_text.shape)
  x,y = get_batch('train')
  print('inputs:')
  print(x)
  print('targets:')
  print(y)

145652 (4,)
145660
tensor([  2659,  59082, 116018, 104520])
torch.Size([4])
inputs:
tensor([[70, 56, 67, 55,  1, 40, 61, 50],
        [55,  1, 48, 54, 48, 56, 61, 10],
        [51,  1, 48,  1, 61, 52, 50, 58],
        [10,  0,  0, 77, 38, 62,  1, 28]], device='cuda:0')
targets:
tensor([[56, 67, 55,  1, 40, 61, 50, 59],
        [ 1, 48, 54, 48, 56, 61, 10,  0],
        [ 1, 48,  1, 61, 52, 50, 58,  8],
        [ 0,  0, 77, 38, 62,  1, 28,  1]], device='cuda:0')


In [ ]:
class BigramLanguageModel(nn.Module):
  # the table is needed for the probas of the different characters w.r.t
  # different characters (proba of a coming after the character c, and so on.)

  def __init__(self,vocab_size):
    super().init_()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, indices, targets = None):
    logits = self.token_embedding_table(indices)
    Batch,Time, Channel = logits.shape
    logits = logits.view(Batch*Time, Channel) # logits should have the same time and the batch
    # The view simply reshapes the output of BTC as the f.cross entropy hasa pecular way of taking in data.
    targets = targets.view(Batch*Time)

    loss = F.cross_entropy(logits, targets) # f is from torch.nn Functional.
    return logits, loss

    def generate(self,index,max_new_tokens):
      for _ in range(max_new_tokens):
        #getting the predications
        logits, loss = self.forward(indices)
        #apply softmax to get probabilities
        probs = F.softmax(logits, dim = -1) # this will give us the data in the format btach, Channel
        # Sample from the distribution
        index_next = torch.multinomial(probs, nun_samples = 1) # )(B,1)
        # Append sampled index to the running sequence
        indices = torch.cat((indices, index_next), dim = 1) # (B, T+1)
      return indices



In [ ]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)